In [13]:
import numpy as np
import pandas as pd
import random
from docplex.mp.model import Model
from tabulate import tabulate

# Parameters
I, J, L, K = 3, 10, 10, 3
Q = [13.5, 15, 18]  # Truck capacities
e = [1339, 1330, 1300]  # Medical shop cost coefficients
f = [1628, 1480, 1563]  # Food shop cost coefficients
THC = [2000, 2500, 2800]  # Truck hiring costs per truck type
DHC = 10000  # Driver hire cost
T_max = 8
service_time = 0.25
travel_time_per_unit = 0.1

# Taguchi L16 array
l16 = [
 (120,0.4,0.04,50),(120,0.5,0.06,100),(120,0.6,0.08,150),(120,0.3,0.10,200),
 (60,0.4,0.06,150),(60,0.5,0.04,200),(60,0.6,0.10,50),(60,0.3,0.08,100),
 (80,0.4,0.08,200),(80,0.5,0.10,150),(80,0.6,0.04,100),(80,0.3,0.06,50),
 (100,0.4,0.10,100),(100,0.5,0.08,50),(100,0.6,0.06,200),(100,0.3,0.04,150)
]

# Random demands
demand_med = np.random.randint(5, 15, size=J)
demand_food = np.random.randint(5, 15, size=L)

# ------------------ CPLEX MILP ------------------
def solve_cplex(d_med, d_food):
    mdl = Model(name="HTFO_CPLEX")
    xjk = [[[mdl.binary_var(name=f"xjk_{j}_{k}_{t}") for t in range(10)] for k in range(K)] for j in range(J)]
    xlk = [[[mdl.binary_var(name=f"xlk_{l}_{k}_{t}") for t in range(10)] for k in range(K)] for l in range(L)]
    ykt = [[mdl.binary_var(name=f"ykt_{k}_{t}") for t in range(10)] for k in range(K)]

    for j in range(J):
        mdl.add_constraint(mdl.sum(xjk[j][k][t] for k in range(K) for t in range(10)) == 1)
    for l in range(L):
        mdl.add_constraint(mdl.sum(xlk[l][k][t] for k in range(K) for t in range(10)) == 1)

    total_cost = 0
    for k in range(K):
        for t in range(10):
            time_expr = 0
            load_expr = 0
            for j in range(J):
                u = d_med[j]
                time_expr += (service_time + travel_time_per_unit * u) * xjk[j][k][t]
                load_expr += u * xjk[j][k][t]
                total_cost += e[k] * u * xjk[j][k][t]
                mdl.add_constraint(ykt[k][t] >= xjk[j][k][t])
            for l in range(L):
                u = d_food[l]
                time_expr += (service_time + travel_time_per_unit * u) * xlk[l][k][t]
                load_expr += u * xlk[l][k][t]
                total_cost += f[k] * u * xlk[l][k][t]
                mdl.add_constraint(ykt[k][t] >= xlk[l][k][t])
            mdl.add_constraint(time_expr <= T_max)
            mdl.add_constraint(load_expr <= Q[k])
            total_cost += THC[k] * ykt[k][t]

    driver_count = mdl.sum(ykt[k][t] for k in range(K) for t in range(10))
    mdl.minimize(total_cost + DHC * driver_count)
    sol = mdl.solve()
    return mdl.objective_value if sol else None

# ------------------ Genetic Algorithm ------------------
def ga_cost(assign, d_med, d_food):
    total = 0
    time_viol_penalty = 1e6
    load_viol_penalty = 1e6
    time_per_driver = [0 for _ in range(K * 10)]
    load_per_driver = [0 for _ in range(K * 10)]
    assign_count = [0] * (K * 10)

    for j, (k, t) in enumerate(assign[:J]):
        idx = k * 10 + t
        total += e[k] * d_med[j]
        time_per_driver[idx] += service_time + travel_time_per_unit * d_med[j]
        load_per_driver[idx] += d_med[j]
        assign_count[idx] += 1

    for l, (k, t) in enumerate(assign[J:]):
        idx = k * 10 + t
        total += f[k] * d_food[l]
        time_per_driver[idx] += service_time + travel_time_per_unit * d_food[l]
        load_per_driver[idx] += d_food[l]
        assign_count[idx] += 1

    for i in range(K * 10):
        if assign_count[i] > 0:
            total += DHC + THC[i // 10]
        if time_per_driver[i] > T_max:
            total += time_viol_penalty
        if load_per_driver[i] > Q[i // 10]:
            total += load_viol_penalty

    return total

def run_ga(d_med, d_food, pop, cxpb, mutpb, gen_max):
    n = J + L
    popu = [[(random.randint(0, K-1), random.randint(0, 9)) for _ in range(n)] for _ in range(pop)]

    for _ in range(gen_max):
        fitness = [ga_cost(ind, d_med, d_food) for ind in popu]
        new_pop = []
        while len(new_pop) < pop:
            a, b, c = random.sample(range(pop), 3)
            p1 = popu[min([(fitness[a], a), (fitness[b], b), (fitness[c], c)])[1]]
            a, b, c = random.sample(range(pop), 3)
            p2 = popu[min([(fitness[a], a), (fitness[b], b), (fitness[c], c)])[1]]
            if random.random() < cxpb:
                pt = random.randint(1, n - 2)
                c1, c2 = p1[:pt] + p2[pt:], p2[:pt] + p1[pt:]
            else:
                c1, c2 = p1[:], p2[:]
            for c in (c1, c2):
                for i in range(n):
                    if random.random() < mutpb:
                        c[i] = (random.randint(0, K-1), random.randint(0, 9))
                new_pop.append(c)
        popu = new_pop[:pop]
    final_fitness = [ga_cost(ind, d_med, d_food) for ind in popu]
    idx = np.argmin(final_fitness)
    return final_fitness[idx], popu[idx]

# ------------------ Run All ------------------
cplex_cost = solve_cplex(demand_med, demand_food)
print(f"\n[CPLEX] Optimal cost: {cplex_cost:.2f}" if cplex_cost else "\n[CPLEX] No feasible solution")

# Taguchi L16 GA Run
l16_results = []
for pop, cx, mut, it in l16:
    costs = []
    for seed in range(3):
        np.random.seed(42 + seed)
        d_med = np.random.randint(5, 15, size=J)
        d_food = np.random.randint(5, 15, size=L)
        cost, _ = run_ga(d_med, d_food, pop, cx, mut, it)
        costs.append(cost)
    avg_cost = np.mean(costs)
    l16_results.append((pop, cx, mut, it, avg_cost))

l16_df = pd.DataFrame(l16_results, columns=['population', 'crossover', 'mutation', 'iterations', 'avg_cost'])
print("\n[L16 TABLE] GA average cost per configuration:")
print(tabulate(l16_df, headers='keys', tablefmt='github'))

best_params = l16_df.sort_values('avg_cost').iloc[0]
print(f"\n[SELECTED GA CONFIG] Best GA config: {best_params.to_dict()}")

final_cost, final_assign = run_ga(demand_med, demand_food,
                                  int(best_params['population']),
                                  best_params['crossover'],
                                  best_params['mutation'],
                                  int(best_params['iterations']))

if cplex_cost:
    print(f"\n[FINAL GA] Cost with best L16 config: {final_cost:.2f}")
    print(f"[COMPARISON] GA cost = {final_cost:.2f}, CPLEX cost = {cplex_cost:.2f}, Deviation = {(final_cost - cplex_cost)/cplex_cost*100:.2f}%")
else:
    print(f"\n[FINAL GA] Cost with best L16 config: {final_cost:.2f}")
    print("[COMPARISON] CPLEX was infeasible.")



[CPLEX] Optimal cost: 426219.00

[L16 TABLE] GA average cost per configuration:
|    |   population |   crossover |   mutation |   iterations |   avg_cost |
|----|--------------|-------------|------------|--------------|------------|
|  0 |          120 |         0.4 |       0.04 |           50 |     427691 |
|  1 |          120 |         0.5 |       0.06 |          100 |     430585 |
|  2 |          120 |         0.6 |       0.08 |          150 |     454089 |
|  3 |          120 |         0.3 |       0.1  |          200 |     456548 |
|  4 |           60 |         0.4 |       0.06 |          150 |     428762 |
|  5 |           60 |         0.5 |       0.04 |          200 |     747998 |
|  6 |           60 |         0.6 |       0.1  |           50 |     455230 |
|  7 |           60 |         0.3 |       0.08 |          100 |     447336 |
|  8 |           80 |         0.4 |       0.08 |          200 |     453654 |
|  9 |           80 |         0.5 |       0.1  |          150 |     4707